In [1]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import utils.feature_extractors as utils
import optuna
import joblib

from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from utils.utils import evaluate_classification
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier

d:\Academic\Semester 8\Bio Informatics\druggable-protien-detection\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_map = {
    "train_postive_location":"dataset/TR_pos_SPIDER.txt",
    "train_negative_location":"dataset/TR_neg_SPIDER.txt",
    "test_positive_location":"dataset/TS_pos_SPIDER.txt",
    "test_negative_location":"dataset/TS_neg_SPIDER.txt",
    "model_save_location":"./optimized_models",
    "feature_combo_model_save_location":"./feature_combo_models",
    "random_seed":9
}

In [3]:
class ProteinFeatureGenerator:
    SELECTED_FEATURES = ["AAC", "DPC", "RScharge", "RSDHP", "RSpolar"]
    
    def __init__(self, positive_data_file: str, negative_data_file: str, feature_type: str = None) -> None:
        super().__init__()

        # Check feature param
        assert feature_type in ProteinFeatureGenerator.SELECTED_FEATURES or feature_type is None
        self.feature_type = feature_type

        # Data manipulation
        self.positive_data_file = positive_data_file
        self.negative_data_file = negative_data_file

        self.positive_data = utils.read_fasta(self.positive_data_file)
        self.negative_data = utils.read_fasta(self.negative_data_file)
        
        self.data = self.positive_data + self.negative_data
        self.targets = np.array([True]*len(self.positive_data) + [False]*len(self.negative_data))

        self.raw_sequences = [x[1] for x in self.data]
        
        
        # Extract features
        print("Extracting AAC Feature ...")
        self.AAC_feature = utils.AAC(self.data)[0]

        print("Extracting DPC Feature ...")
        self.DPC_feature = utils.DPC(self.data, 0)[0]

        print("Extracting RScharge Feature ...")
        self.RScharge_feature = utils.reducedCHARGE(self.data)
        
        print("Extracting RSDHP Feature ...")
        self.RSDHP_feature = utils.reducedDHP(self.data)
        
        print("Extracting RSpolar Feature ...")
        self.RSpolar_feature = utils.reducedPOLAR(self.data)
        
    
    def __len__(self) -> int:
        return len(self.data)

In [4]:
train_data = ProteinFeatureGenerator(positive_data_file=config_map["train_postive_location"],negative_data_file=config_map["train_negative_location"])

Extracting AAC Feature ...
Extracting DPC Feature ...
Extracting RScharge Feature ...
Extracting RSDHP Feature ...
Extracting RSpolar Feature ...


In [5]:
X_train = {
    "AAC":train_data.AAC_feature,
    "DPC":train_data.DPC_feature,
    "RScharge":train_data.RScharge_feature,
    "RSDHP":train_data.RSDHP_feature,
    "RSpolar":train_data.RSpolar_feature,
}

feature_model_map = {
    "AAC":"SVC",
    "DPC":"SVC",
    "RScharge":"RandomForest",
    "RSDHP":"SVC",
    "RSpolar":"ExtraTreesClassifier",
}

feature_combos = ["AAC-DPC-RScharge-RSDHP-RSpolar",
                  "AAC-RScharge-RSDHP-RSpolar",
                  "DPC-RScharge-RSDHP-RSpolar",
                  "AAC-DPC-RScharge",
                  "AAC-DPC-RSDHP",
                  "AAC-DPC-RSpolar",
                  "RScharge-RSDHP-RSpolar",
                  "AAC-RSpolar",
                  ]

In [6]:
best_feature_combo = ""
best_accuracy = 0
best_clf = None
# Loop over all feature combinations
for feature_combination in feature_combos:
    
    # Initialize classifier model and its string representation
    clf = SVC()
    clf_name = "SVC"
    
    # Print model and feature combination
    print(f"Feature Combination: {feature_combination} | Classifier Model: {clf}")
    
    # Create directories for model storage
    os.makedirs(os.path.join(config_map["feature_combo_model_save_location"], feature_combination, clf_name), exist_ok=True)
    model_dir = os.path.join(config_map["feature_combo_model_save_location"], feature_combination, clf_name)
    
    # Train the model and obtain probabilities using cross validation
    probabilities = []
    for feature_type in feature_combination.split("-"):
        pipeline = joblib.load(os.path.join(config_map["model_save_location"], feature_type, feature_model_map[feature_type], "pipeline.sav"))
        feat_clf = joblib.load(os.path.join(config_map["model_save_location"], feature_type, feature_model_map[feature_type], "model.sav"))
        
        # Transform data using pipeline and obtain decision function probabilities
        X = X_train[feature_type]
        X = pipeline.transform(X)
        if feature_model_map[feature_type] == "RandomForest" or feature_model_map[feature_type] == "ExtraTreesClassifier":
            y_pred = feat_clf.predict_proba(X)[:, 1].reshape(-1, 1)
        else:
            y_pred = feat_clf.decision_function(X).reshape(-1,1)
        
        # Append probabilities to list
        probabilities.append(y_pred)
     
    # Concatenate probabilities and obtain predictions using cross validation
    probabilities = np.concatenate(probabilities, axis=-1)
    y_pred = cross_val_predict(clf, probabilities, train_data.targets, cv=5)
    
    # Evaluate results and save model
    result_values = evaluate_classification(y_pred, train_data.targets, class_names=["Not Druggable","Druggable"], save_outputs=model_dir)
    clf.fit(probabilities, train_data.targets)
    

    joblib.dump(clf, os.path.join(model_dir, "model.sav"))


    #Update top values
    if float(result_values["accuracy"])>best_accuracy:
        best_accuracy = result_values["accuracy"]
        best_feature_combo = feature_combination
        best_clf = clf
    
    # Print results and new line characters
    print(result_values)
    print("\n")

print("====================================")
print("Best Feature Combo: "+ best_feature_combo)
print("Best Accuracy: "+ str(best_accuracy))

Feature Combination: AAC-DPC-RScharge-RSDHP-RSpolar | Classifier Model: SVC()
{'accuracy': 0.993705743509048, 'sensitivity': 0.9926410466067048, 'specificity': 0.9946929492039424, 'precision': 0.9937269869139664, 'f1': 0.9936963745474384}


Feature Combination: AAC-RScharge-RSDHP-RSpolar | Classifier Model: SVC()
{'accuracy': 0.99213217938631, 'sensitivity': 0.9885527391659853, 'specificity': 0.9954510993176648, 'precision': 0.9922558076790682, 'f1': 0.9921189877828809}


Feature Combination: DPC-RScharge-RSDHP-RSpolar | Classifier Model: SVC()
{'accuracy': 0.9933123524783635, 'sensitivity': 0.9926410466067048, 'specificity': 0.9939347990902199, 'precision': 0.9933175866686506, 'f1': 0.9933026003683927}


Feature Combination: AAC-DPC-RScharge | Classifier Model: SVC()
{'accuracy': 0.993705743509048, 'sensitivity': 0.9926410466067048, 'specificity': 0.9946929492039424, 'precision': 0.9937269869139664, 'f1': 0.9936963745474384}


Feature Combination: AAC-DPC-RSDHP | Classifier Model: SVC